# Acompanhamento de Ações na B3

# Install Dependencies

In [0]:
!pip install yfinance

# Imports

In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import pyspark
from pyspark.sql import DataFrame, SparkSession

# Configure Widgets and Variables

In [0]:
# Definir data atual
data_atual = datetime.now().strftime("%Y-%m-%d")

In [0]:
# Criar Widgets
dbutils.widgets.text("1. Código Ações", "")
dbutils.widgets.text("2. Data Inicial (Ano-Mês-Dia)", data_atual, "")
dbutils.widgets.text("3. Data Final (Ano-Mês-Dia)", data_atual, "")

In [0]:
# Obter valores dos widgets
filter_acoes = dbutils.widgets.get("1. Código Ações")
data_inicial = dbutils.widgets.get("2. Data Inicial (Ano-Mês-Dia)")
data_final = dbutils.widgets.get("3. Data Final (Ano-Mês-Dia)")

In [0]:
# Tratar valores e filtrar médias móveis
acoes = filter_acoes.split(",")
filter_media_movel = ["15", "30", "60", "90", "120"]

# Get Stock Price and Moving Averages

In [0]:
# Inicializar uma lista para os dados de cada ação
dados_acoes = []

for acao in acoes:
    print(acao)
    tabela_cotacoes = yf.download(acao, start=data_inicial, end=data_final)

    # Adicionar código da ação à tabela de cotações
    tabela_cotacoes["Code"] = acao
    tabela_cotacoes = tabela_cotacoes.reset_index()

    # Calcular médias móveis e adicionar à tabela de cotações
    for media_movel in filter_media_movel:
        tabela_cotacoes[f"Media_movel_{media_movel}d"] = tabela_cotacoes["Close"].rolling(int(media_movel)).mean()

    # Adicionar a tabela atual à lista de dados
    dados_acoes.append(tabela_cotacoes)

# Concatenar todas as tabelas acumuladas em um único DataFrame
df_acoes = pd.concat(dados_acoes, ignore_index=True)

# Exibir o DataFrame final
display(df_acoes)


## Create Temporary View for Stock Analysis

In [0]:
spark.createDataFrame(df_acoes).createOrReplaceTempView("acoes_analise_temp")

## Define Columns for Analysis and Create View

In [0]:
colunas = """
CAST(Date AS Date) as data_negociacao, 
Code as codigo, 
round(Open, 2) as abertura, 
round(Close, 2) as fechamento, 
round(Close - Open, 4) as diff, 
round(High, 2) as maior_preco, 
round(Low, 2) as menor_preco, 
Volume as volume_necociacoes
"""

for media_movel in filter_media_movel:
    colunas += f", round(Media_Movel_{media_movel}d, 2) as media_movel_{media_movel}d"

acoes_analise = spark.sql(f"SELECT {colunas} FROM acoes_analise_temp ORDER BY data_negociacao")

acoes_analise.createOrReplaceTempView("acoes_analise")
display(acoes_analise)


# Get Dividend Data

In [0]:
# Inicializar uma lista para acumular os dados de dividendos de cada ação
dados_dividendos = []

for acao in acoes:
    tabela_dividendos = yf.Ticker(acao).history(start=data_inicial, end=data_final)

    # Adicionar código da ação à tabela de dividendos
    tabela_dividendos["Code"] = acao
    tabela_dividendos = tabela_dividendos.reset_index()

    # Adicionar a tabela atual à lista de dados
    dados_dividendos.append(tabela_dividendos)

# Concatenar todas as tabelas acumuladas em um único DataFrame
df_dividendos_temp = pd.concat(dados_dividendos, ignore_index=True)

# Exibir o DataFrame final
display(df_dividendos_temp)


## Create Temporary View for Dividends

In [0]:
spark.createDataFrame(df_dividendos_temp).createOrReplaceTempView("dividendos_temp")

## Define Columns for Dividend Analysis and Create View

In [0]:
colunas_dividendos = """
CAST(Date AS Date) as data_negociacao, 
Code as codigo, 
round(Open, 2) as abertura, 
round(Close, 2) as fechamento, 
round(Dividends, 6) as dividendos
"""

df_dividendos = spark.sql(f"SELECT {colunas_dividendos} FROM dividendos_temp ORDER BY data_negociacao")

df_dividendos.createOrReplaceTempView("df_dividendos")

display(df_dividendos)


# SQL Queries for Data Analysis

## Daily Performance

In [0]:
%sql

SELECT
  data_negociacao, 
  codigo, 
  abertura, 
  fechamento, 
  maior_preco as maxima,
  menor_preco as minima,
  round(diff, 2) AS diff
FROM acoes_analise
ORDER BY data_negociacao DESC

## Latest Profit/Loss

In [0]:
%sql

SELECT
  codigo,
  ANY_VALUE(abertura) AS abertura,
  ANY_VALUE(fechamento) AS fechamento,
  round(diff, 2) as diff
FROM acoes_analise
WHERE 
  data_negociacao = (
    SELECT max(data_negociacao)
    FROM acoes_analise
  )
GROUP BY codigo, diff


## Daily Variation

In [0]:
%sql

WITH 
datas as (
  SELECT
    max(data_negociacao) as data_atual,
    (SELECT max(data_negociacao) FROM acoes_analise WHERE data_negociacao < (SELECT max(data_negociacao) FROM acoes_analise)) AS data_anterior
  FROM acoes_analise
),
pega_valores as (
SELECT
  acoes.codigo,
  CASE WHEN acoes.data_negociacao = datas.data_atual THEN acoes.fechamento END as preco_atual,
  CASE WHEN acoes.data_negociacao = datas.data_anterior THEN acoes.fechamento END as preco_anterior  
FROM acoes_analise acoes
INNER JOIN datas ON datas.data_atual = acoes.data_negociacao OR datas.data_anterior = acoes.data_negociacao
)
SELECT
  pega_valores.codigo,
  max(preco_atual) as preco_atual, 
  max(preco_anterior) as preco_anterior, 
  round(((max(preco_atual) - max(preco_anterior)) / (max(preco_anterior))) * 100, 2) AS perc_variacao_diaria
FROM pega_valores
GROUP BY pega_valores.codigo


## Annual Profit Summary by Stock

In [0]:
%sql

WITH 
datas as (
  SELECT
    codigo,
    min(data_negociacao) as primeira_cotacao_ano,
    max(data_negociacao) as ultima_cotacao_ano
  FROM acoes_analise
  WHERE year(data_negociacao) = year(current_date())
  GROUP BY codigo
),
pega_valores as (
  SELECT
    acoes.codigo,
    CASE WHEN acoes.data_negociacao = datas.primeira_cotacao_ano THEN acoes.abertura ELSE 0 END as abertura,
    CASE WHEN acoes.data_negociacao = datas.ultima_cotacao_ano THEN acoes.fechamento ELSE 0 END as fechamento
  FROM acoes_analise acoes
  INNER JOIN datas ON datas.codigo = acoes.codigo
    AND (acoes.data_negociacao = datas.primeira_cotacao_ano OR acoes.data_negociacao = datas.ultima_cotacao_ano)
)
SELECT
  codigo,
  sum(abertura) as abertura, 
  sum(fechamento) as fechamento, 
  round(sum(fechamento) - sum(abertura), 2) as lucro_ano
FROM pega_valores
GROUP BY codigo


## YTD Price (Year-to-Date Price)

In [0]:
%sql

SELECT
  data_negociacao, 
  month(data_negociacao) as mes, 
  codigo, 
  abertura, 
  fechamento
FROM acoes_analise
WHERE year(data_negociacao) = year(current_date())
ORDER BY data_negociacao

## Trading Volume

In [0]:
%sql

WITH
volume_total as (
  SELECT
    codigo,
    volume_necociacoes as volume_total_ultimo_dia 
  FROM 
    acoes_analise 
  WHERE 
    data_negociacao = (
      SELECT max(data_negociacao)
      FROM acoes_analise
    )
),

media_volume as (
  SELECT
    codigo,
    round(AVG(volume_necociacoes), 2) AS media_volume_ultimos_30_dias
  FROM 
    acoes_analise
  WHERE 
    data_negociacao >= CURRENT_DATE - INTERVAL '30 days'
  GROUP BY
    codigo
)

SELECT
  media_volume.codigo,
  volume_total.volume_total_ultimo_dia,
  media_volume.media_volume_ultimos_30_dias
FROM
  media_volume
INNER JOIN volume_total ON volume_total.codigo = media_volume.codigo


## Dividend Yield

In [0]:
%sql

WITH
valor_acao as (
  SELECT 
    codigo,
    fechamento as preco_atual  
  FROM 
    acoes_analise
  WHERE 
    data_negociacao = (
      SELECT MAX(data_negociacao)
      FROM acoes_analise
    )
),

soma_dividendos as (
  SELECT 
    codigo,
    round(SUM(dividendos), 2) as total_dividendo_365
  FROM 
    df_dividendos
  WHERE 
    data_negociacao >= CURRENT_DATE - INTERVAL '365 days'
  GROUP BY 
    codigo
)

SELECT
  valor_acao.codigo,
  soma_dividendos.total_dividendo_365,
  valor_acao.preco_atual,
  round((soma_dividendos.total_dividendo_365 / valor_acao.preco_atual) * 100, 2) as dividend_yield
FROM 
  soma_dividendos
INNER JOIN 
  valor_acao ON soma_dividendos.codigo = valor_acao.codigo

## Dividends YTD vs Previous Year

In [0]:
%sql
SELECT
  codigo,
  round(SUM(
    CASE WHEN YEAR(data_negociacao) = YEAR(CURRENT_DATE()) 
    THEN dividendos 
    ELSE 0 
    END
  ), 2
) as total_dividendos_ano_atual,
  round(SUM(
    CASE WHEN YEAR(data_negociacao) = YEAR(CURRENT_DATE()) - 1 
    THEN dividendos 
    ELSE 0 
    END
  ), 2
) AS total_dividendos_ultimo_ano
FROM 
  df_dividendos
GROUP BY codigo


## Dividends Paid by Month/Year

In [0]:
%sql

SELECT
  codigo,
  sum(dividendos) as soma_dividendos,
  month(data_negociacao) as mes,
  year(data_negociacao) as ano,
  concat(month(data_negociacao), "/", year(data_negociacao)) as ref
FROM df_dividendos
WHERE (
  data_negociacao BETWEEN date_trunc('YEAR', date_add(current_date(), -365))
  AND date_add(current_date() -1, -366)
  OR year(data_negociacao) = year(current_date())
)
  AND dividendos > 0
GROUP BY codigo, mes, ano
ORDER BY ano, mes

# Get Infos from Finnhub

## Install Dependencies

In [0]:
!pip install finnhub-python

## Imports

In [0]:
import finnhub

## Get Stock Infos

In [0]:
# Insira sua chave de API da Finnhub
API_KEY = ""

# Inicialize o cliente da API
finnhub_client = finnhub.Client(api_key=API_KEY)

# Símbolo da ação para a qual deseja obter informações
symbol = ""

# Obtenha o perfil da empresa
profile = finnhub_client.company_profile2(symbol=symbol)

nome_empresa = profile.get("name", "Não Disponível")
setor = profile.get("finnhubIndustry", "Não Disponível")
industria = profile.get("industry", "Não Disponível")
pais = profile.get("country", "Não Disponível")
moeda = profile.get("currency", "Não Disponível")

# Exiba as informações principais
print(f"Nome da Empresa: {nome_empresa}")
print(f"Setor: {setor}")
print(f"Indústria: {industria}")
print(f"País: {pais}")
print(f"Moeda: {moeda}")
